# Environmental Setup


Install necessary modules and clone the Github repo.

In [ ]:
!pip install wget

import os
import shutil
import wget
from urllib.parse import urlparse

%cd /content/

repo_path = "https://github.com/KyawYeThu-11/DyGLib.git"
repo_name = os.path.splitext(os.path.basename(urlparse(repo_path).path))[0]

if not os.path.exists(repo_name):
  !git clone $repo_path
  !pip install -r DyGLib/requirements.txt

In [ ]:
%cd /content/DyGLib

/content/DyGLib


In [ ]:
import logging
import time
import sys
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import warnings
import shutil
import json
import torch
import torch.nn as nn
from torch_geometric.nn import global_mean_pool

from models.DyGFormerGraph import DyGFormerGraph
from models.DyGFormer import DyGFormer
from models.modules import MergeLayer, GraphRegressor
from utils.utils import set_random_seed, convert_to_gpu, get_parameter_sizes, create_optimizer
from utils.utils import get_neighbor_sampler, NegativeEdgeSampler
from utils.metrics import get_link_prediction_metrics
from utils.DataLoader import get_idx_data_loader, get_graph_regression_data
from utils.EarlyStopping import EarlyStopping
from utils.load_configs import get_link_prediction_args

# Training

### Constants and Helper Functions

In [ ]:
class Args:
    def __init__(self, dataset_name):
        self.dataset_name = dataset_name
        self.batch_size = 1000
        self.model_name = 'DyGFormer'
        self.gpu = 0
        self.num_neighbors = 20
        self.sample_neighbor_strategy = 'recent'
        self.time_scaling_factor = 1e-6
        self.num_walk_heads = 8
        self.num_folds = 5
        self.num_heads = 2
        self.num_layers = 2
        self.walk_length = 1
        self.time_gap = 2000
        self.time_feat_dim = 100
        self.position_feat_dim = 172
        self.edge_bank_memory_mode = 'unlimited_memory'
        self.time_window_mode = 'fixed_proportion'
        self.patch_size = 2
        self.channel_embedding_dim = 50
        self.max_input_sequence_length = 64
        self.learning_rate = 0.005
        self.load_checkpoint = False
        self.dropout = 0.05
        self.scheduler_period = 2
        self.num_epochs = 10
        self.optimizer = 'Adam'
        self.weight_decay = 0.0
        self.patience = 20
        self.test_interval_epochs = 10
        self.negative_sample_strategy = 'random'
        self.load_best_configs = False

    def __str__(self):
        properties = [f"{key}={value}" for key, value in self.__dict__.items()]
        return f"Args({', '.join(properties)})"

    def __repr__(self):
        return self.__str__()

# Create an instance of Args with the loaded configuration
args = Args(dataset_name = 'connectome')
args.device = f'cuda:{args.gpu}' if torch.cuda.is_available() and args.gpu >= 0 else 'cpu'


In [ ]:
def set_up_logger(args):
        # set up logger
        logging.basicConfig(level=logging.INFO)
        logger = logging.getLogger()
        logger.setLevel(logging.DEBUG)
        os.makedirs(f"./logs/{args.model_name}/{args.dataset_name}/", exist_ok=True)
        # create file handler that logs debug and higher level messages
        fh = logging.FileHandler(f"./logs/{args.model_name}/{args.dataset_name}/{str(time.time())}.log")
        fh.setLevel(logging.DEBUG)
        # create console handler with a higher log level
        ch = logging.StreamHandler()
        ch.setLevel(logging.WARNING)
        # create formatter and add it to the handlers
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        fh.setFormatter(formatter)
        ch.setFormatter(formatter)
        # add the handlers to logger
        logger.addHandler(fh)
        logger.addHandler(ch)

        return logger, fh, ch

In [ ]:
def step_forward(dataset_dir, index, subject_id, label, loss_func, mode):
  node_raw_features, edge_raw_features, full_data = \
        get_graph_regression_data(dataset_dir=dataset_dir, index=index)

  idx_data_loader = get_idx_data_loader(indices_list=list(range(len(full_data.src_node_ids))), batch_size=args.batch_size, shuffle=False)
  idx_data_loader_tqdm = tqdm(idx_data_loader, ncols=120)

  # initialize validation and test neighbor sampler to retrieve temporal graph
  full_neighbor_sampler = get_neighbor_sampler(data=full_data, sample_neighbor_strategy=args.sample_neighbor_strategy,
                                                  time_scaling_factor=args.time_scaling_factor, seed=1)
  # training, only use training graph
  model[0].set_neighbor_sampler(full_neighbor_sampler)

  graph_embedding_list = []

  for batch_idx, train_data_indices in enumerate(idx_data_loader_tqdm):


    train_data_indices = train_data_indices.numpy()
    batch_src_node_ids, batch_dst_node_ids, batch_node_interact_times, batch_edge_ids = \
                      full_data.src_node_ids[train_data_indices], full_data.dst_node_ids[train_data_indices], \
                      full_data.node_interact_times[train_data_indices], full_data.edge_ids[train_data_indices]

    graph_embedding = model[0](node_raw_features, edge_raw_features, batch_src_node_ids, batch_dst_node_ids, batch_node_interact_times)
    graph_embedding_list.append(graph_embedding)

    if mode == 'train':
      idx_data_loader_tqdm.set_description(f'Epoch: {epoch + 1}, Subject: {subject_id}, training for the {batch_idx + 1}-th batch.')

  graph_embeddings_tensor = torch.tensor(np.stack(graph_embedding_list).squeeze()).to(args.device)

  predict = model[1](graph_embeddings_tensor)
  loss = loss_func(input=predict, target=label)

  return predict, loss

### Main

In [ ]:
percentile_folder = '5-percentile'
save_model_folder = f"saved_models/graph_regression/{args.dataset_name}"
os.makedirs(save_model_folder, exist_ok=True)
checkpoint_path = os.path.join(save_model_folder, f'{args.model_name}.pth')

logger, fh, ch = set_up_logger(args)

tart_time = time.time()

logger.info(f'configuration is {args}')

# Initialize the model
dynamic_backbone = DyGFormerGraph(node_feat_dim=args.position_feat_dim, edge_feat_dim=args.position_feat_dim, time_feat_dim=args.time_feat_dim,
                                 channel_embedding_dim=args.channel_embedding_dim, patch_size=args.patch_size,
                                 num_layers=args.num_layers, num_heads=args.num_heads, dropout=args.dropout,
                                 max_input_sequence_length=args.max_input_sequence_length, device=args.device)

graph_regressor = GraphRegressor(in_channels=args.position_feat_dim, hidden_channels=int(args.position_feat_dim/2), out_channels=int(args.position_feat_dim/4))
model = nn.Sequential(dynamic_backbone, graph_regressor)

# log the model structure
logger.info(f'model -> {model}')
logger.info(f'model name: {args.model_name}, #parameters: {get_parameter_sizes(model) * 4} B, '
                    f'{get_parameter_sizes(model) * 4 / 1024} KB, {get_parameter_sizes(model) * 4 / 1024 / 1024} MB.')

# Create the optimizer and scheduler with specified parameters
optimizer = create_optimizer(model=model, optimizer_name=args.optimizer, learning_rate=args.learning_rate, weight_decay=args.weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.scheduler_period)

# Convert the model to GPU if available
model = convert_to_gpu(model, device=args.device)


epoch_resumed = 0
index_resumed = 0
# Load checkpoint if specified and exists
if args.load_checkpoint == True and os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=args.device)
    logger.info(f"load model {checkpoint_path}")

    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    scheduler.load_state_dict(checkpoint['scheduler'])

    epoch_resumed = checkpoint['epoch']
    index_resumed = checkpoint['subject_index'] + 1
    print(f"Epoch resumed: {epoch_resumed}")
    print(f"Index resumed: {index_resumed}")

# Define the mean square loss function
loss_func = nn.MSELoss()

# Training loop
for epoch in range(epoch_resumed, args.num_epochs, 1):

    # Training for an epoch starts
    model.train()

    train_losses = []

    language_score_df = pd.read_csv(f'DG_data/{args.dataset_name}/{percentile_folder}/Train_Data_csv/Language_Task_Acc.csv')

    # Iterate over each subject in the training data
    for index, row in language_score_df.iterrows():
        if index < index_resumed:
          continue
        subject_id = int(row.iloc[2])
        score = torch.Tensor([row.iloc[3]]).to(args.device)
        subject_folder = os.path.join(f'processed_data/{args.dataset_name}/{percentile_folder}/Train', str(index))

        # process every batch for a single subject and compute predictions and loss
        predict, loss = step_forward(subject_folder, index, subject_id, score, loss_func, 'train')

        train_losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        print(f"For subject {subject_id}, label: {score.item():.4f}, predict: {predict.item():.4f}, and loss: {loss:.4f}")

        # Save the checkpoint
        torch.save({
                  'subject_index': index,
                  'epoch': epoch,
                  'model': model.state_dict(),
                  'optimizer': optimizer.state_dict(),
                  'scheduler': scheduler.state_dict()},
                  checkpoint_path)

    # validation
    print("---------------Validation Starts--------------")

    model.eval()

    val_losses = []

    with torch.no_grad():
        language_score_df = pd.read_csv(f'DG_data/{args.dataset_name}/{percentile_folder}/Val_Data_csv/Language_Task_Acc.csv')

        for index, row in language_score_df.iterrows():
            subject_id = int(row.iloc[2])
            score = torch.Tensor([row.iloc[3]]).to(args.device)
            subject_folder = os.path.join(f'processed_data/{args.dataset_name}/{percentile_folder}/Val', str(index))

            predict, loss = step_forward(subject_folder, index, subject_id, score, loss_func, 'test')

            val_losses.append(loss.item())

            print(f"For subject {subject_id}, label: {score.item():.4f}, predict: {predict.item():.4f}, and loss: {loss:.4f}")

    print("-----------------------------------")
    logger.info(f'Epoch: {epoch}, learning rate: {optimizer.param_groups[0]["lr"]}, train loss: {np.mean(train_losses):.4f} and val loss: {np.mean(val_losses):.4f}')
    print("-----------------------------------")

    # Save the results of the current run to a JSON file
    result_json = {
            "train losses": [f'{loss:.4f}' for loss in train_losses],
            'val losses': [f'{loss:.4f}' for loss in val_losses]
    }

    result_json = json.dumps(result_json, indent=4)

    save_result_folder = f"saved_results/graph_regression/{args.dataset_name}"
    os.makedirs(save_result_folder, exist_ok=True)
    save_result_path = os.path.join(save_result_folder, f"{args.save_model_name}.json")

    with open(save_result_path, 'w') as file:
        file.write(result_json)

torch.save(model.state_dict(), os.path.join(save_model_folder, f'{args.model_name}_final.pth'))

## Testing

Load the model.

In [ ]:
# Initialize the model
dynamic_backbone = DyGFormerGraph(node_feat_dim=args.position_feat_dim, edge_feat_dim=args.position_feat_dim, time_feat_dim=args.time_feat_dim,
                                 channel_embedding_dim=args.channel_embedding_dim, patch_size=args.patch_size,
                                 num_layers=args.num_layers, num_heads=args.num_heads, dropout=args.dropout,
                                 max_input_sequence_length=args.max_input_sequence_length, device=args.device)

graph_regressor = GraphRegressor(in_channels=args.position_feat_dim, hidden_channels=int(args.position_feat_dim/2), out_channels=int(args.position_feat_dim/4))
model = nn.Sequential(dynamic_backbone, graph_regressor)

# Load the pretrained model from the specified path
load_model_path = f"saved_models/graph_regression/{args.dataset_name}/DyGFormer.pth"
checkpoint = torch.load(load_model_path, map_location=args.device)
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

Test the model with the testing set.

In [ ]:
percentile_folder = '5-percentile'
test_losses = []
loss_func = nn.MSELoss()

with torch.no_grad():
    language_score_df = pd.read_csv(f'DG_data/{args.dataset_name}/{percentile_folder}/Test_Data_csv/Language_Task_Acc.csv')

    for index, row in language_score_df.iterrows():
        subject_id = int(row.iloc[2])
        score = torch.Tensor([row.iloc[3]]).to(args.device)
        subject_folder = os.path.join(f'processed_data/{args.dataset_name}/{percentile_folder}/Test', str(index))

        predict, loss = step_forward(subject_folder, index, subject_id, score, loss_func, 'test')

        test_losses.append(loss.item())

        print(f"For subject {subject_id}, label: {score.item():.4f}, predict: {predict.item():.4f}, and loss: {loss:.4f}")

    print(f'test loss: {np.mean(test_losses):.4f}')